In [3]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [4]:
from pyquery import PyQuery as pq
from bs4 import BeautifulSoup
import re
import requests

In [3]:
m2015jan = requests.get("http://www.federalreserve.gov/monetarypolicy/fomcminutes20150128.htm")

# PyQuery 

Getting features and dividing structure from a single page

In [4]:
d_ = pq(m2015jan.text)
d_rows = pq(d_('strong'))
len(d_rows)

17

In [5]:
d_rows.contents()

['Annual Organizational Matters', <Element a at 0x1f1ab0e8>, <Element a at 0x1f1ab138>, ' ', "Developments in Financial Markets and the Federal Reserve's Balance Sheet", '-', 'Liftoff Tools and Possible Liftoff Options', 'Staff Review of the Economic Situation', 'Staff Review of the Financial Situation', 'Staff Economic Outlook', "Participants' Views on Current Conditions and the Economic Outlook", "Participants' Discussion of Policy Planning", ' ', 'Committee Policy Action', 'Voting for this action:', 'Voting against this action: ', 'Notation Vote', 'Thomas Laubach', 'Secretary']

# BeautifulSoup

Getting features and dividing structure from a single page

In [6]:
soup = BeautifulSoup(m2015jan.text, "html.parser")
sections = soup.find_all('strong')
sections[0].text

u'Annual Organizational Matters5\xa0'

# All FOMC minutes

In [7]:
fomc_base_url = "http://www.federalreserve.gov"

In [8]:
# find all meeting minutes
allmin = requests.get("http://www.federalreserve.gov/monetarypolicy/fomccalendars.htm")
allminsoup = BeautifulSoup(allmin.text, "html.parser")
dates = allminsoup.find_all('td', attrs={'class': 'minutes'})

# get urls of meeting minutes
page_urls = [date.find('a', text="HTML") for date in dates]
page_urls = filter(lambda x: x, page_urls)
page_urls = [p['href'] for p in page_urls]

In [9]:
# should have 47 urls to get
print len(page_urls)
page_urls[0:5]

47


[u'/monetarypolicy/fomcminutes20150128.htm',
 u'/monetarypolicy/fomcminutes20150318.htm',
 u'/monetarypolicy/fomcminutes20150429.htm',
 u'/monetarypolicy/fomcminutes20150617.htm',
 u'/monetarypolicy/fomcminutes20150729.htm']

In [10]:
import time
from requests.exceptions import ConnectionError

fomc_mins = {}

In [17]:
# may need to run this block multiple times
searched_dates = fomc_mins.keys()

for p in page_urls:
    url = fomc_base_url + p
    date = p[-12:-4]
    
    if date not in searched_dates:
        try:
            page = requests.get(url)
            fomc_mins[date] = page.text
        except ConnectionError as e:
            print "Error ==> ", e, "for", date
    
        time.sleep(1)

In [18]:
# should have 47 meeting minutes
len(fomc_mins.keys())

47

In [303]:
import json
"""
# store the scraped minutes html
mins_html = open("fomc_mins.json", "wb")
json.dump(mins_text, mins_html)
mins_html.close()
"""

In [304]:
# open scraped minutes
with open("fomc_mins.json", "rb") as infile:
    fomc_mins = json.load(infile)

## Minutes before 2009